## Capstone Project - The Battle of Neighborhoods

###### Importing Libraries

In [1]:
import pandas as pd
import pgeocode
import requests
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import numpy as np

###### Set maximum column to be displayed

In [2]:
pd.set_option('display.max_columns',None)

###### Read the data from Web

In [3]:
data=pd.read_html("https://finkode.com/wb/kolkata.html")

###### Convert it to a dataframe

In [4]:
kol_df=pd.DataFrame(data[0])
kol_df.head()

,Post Office,District,Pincode
0,A.J.C.Bose Road S.O,Kolkata,700020
1,Abinash Chaowdhury Lane S.O,Kolkata,700046
2,Ahritola S.O,Kolkata,700005
3,Alambazar S.O,Kolkata,700035
4,Amrita Bazar Partika S.O,Kolkata,700003


###### The dataframe contains multiple entries for a single Pincode, so merging the entries in a single row so that we have unique entry for each Picode

In [5]:
kol_df[kol_df['Pincode']== 700005]

,Post Office,District,Pincode
2,Ahritola S.O,Kolkata,700005
75,Hatkhola S.O,Kolkata,700005


In [6]:
merge_kol_df=kol_df.groupby(['Pincode','District']).agg({'Post Office':', '.join}).reset_index()
merge_kol_df

,Pincode,District,Post Office
0,700001,Kolkata,"Council House Street S.O, Customs House S.O, K..."
1,700002,Kolkata,"Cossipore Gun Factory S.O, Cossipore H.O, Indi..."
2,700003,Kolkata,"Amrita Bazar Partika S.O, Baghbazar S.O, Giris..."
3,700004,Kolkata,"R.G.Kar Medical College S.O, Shyambazar Mail S..."
4,700005,Kolkata,"Ahritola S.O, Hatkhola S.O"
...,...,...,...
76,700095,Kolkata,Golf Green S.O
77,700099,Kolkata,"Kalikapur S.O, Mukundapur S.O"
78,700107,Kolkata,E.K.T S.O
79,700108,Kolkata,ISI PO S.O


###### Find Latitude and Longitude of each pincode area, convert the Pincode to a string before using it.

In [7]:
merge_kol_df.dtypes

Pincode         int64
District       object
Post Office    object
dtype: object

In [8]:
merge_kol_df=merge_kol_df.astype({'Pincode':str})

In [9]:
merge_kol_df.dtypes

Pincode        object
District       object
Post Office    object
dtype: object

In [10]:
Latitude=[];
Longitude=[];
for i in range(0,merge_kol_df.shape[0]):
    geolocator=pgeocode.Nominatim('IN')
    location=geolocator.query_postal_code(merge_kol_df.iloc[i]['Pincode'])
    Latitude.append(location.latitude)
    Longitude.append(location.longitude)
merge_kol_df['Latitude']=Latitude
merge_kol_df['Longitude']=Longitude
merge_kol_df

,Pincode,District,Post Office,Latitude,Longitude
0,700001,Kolkata,"Council House Street S.O, Customs House S.O, K...",22.7101,88.3167
1,700002,Kolkata,"Cossipore Gun Factory S.O, Cossipore H.O, Indi...",22.7101,88.3167
2,700003,Kolkata,"Amrita Bazar Partika S.O, Baghbazar S.O, Giris...",22.7101,88.3167
3,700004,Kolkata,"R.G.Kar Medical College S.O, Shyambazar Mail S...",22.7101,88.3167
4,700005,Kolkata,"Ahritola S.O, Hatkhola S.O",22.7101,88.3167
...,...,...,...,...,...
76,700095,Kolkata,Golf Green S.O,22.4485,88.3883
77,700099,Kolkata,"Kalikapur S.O, Mukundapur S.O",22.5553,88.3558
78,700107,Kolkata,E.K.T S.O,22.9660,88.2036
79,700108,Kolkata,ISI PO S.O,22.9660,88.2036


###### Set Client parameters and update the URL string for one coordinate to check whether the foursquare can find out the nearest venues

In [16]:
CLIENT_ID = 'JUJNYLJMT5FGA0KG0TJT4BJ0PFVI3L3DJ3POUJPK5KDII1RP'
CLIENT_SECRET = 'JM2D1DRIVNOLXPWOZYJSI5YIDKTYCV55SQR31HLHBS1YVZNG'
VERSION = '20180605'
limit = 10
radius = 2000
latitude=merge_kol_df.iloc[0]['Latitude']
longitude=merge_kol_df.iloc[0]['Longitude']
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,latitude,longitude,radius,limit)

In [17]:
results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fd2e288e957890549ec9f9c'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 22.728100018000017,
    'lng': 88.33617641545679},
   'sw': {'lat': 22.692099981999984, 'lng': 88.2972235845432}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5c39c1b63abcaf002cf1f18f',
       'name': 'Axis Bank ATM',
       'location': {'address': 'Samir Ghosh',
        'crossStreet': 'Kanaipur Bazar',
        'lat': 22.70056,
        'lng': 88.32262,
        'labeledLatLn

In [18]:
def get_category(data):
    category_type=data['venue.categories']
    if len(category_type)==0:
        return None
    else:
        return category_type[0]['name']

In [19]:
venues=results['response']['groups'][0]['items']
nearby_venues=json_normalize(venues)
nearby_venues=nearby_venues.loc[:,['venue.name','venue.categories','venue.location.lat','venue.location.lng']]
nearby_venues['venue.categories']=nearby_venues.apply(get_category,axis=1)
nearby_venues.columns=[column.split(".")[-1] for column in nearby_venues.columns]
nearby_venues.head()

C:\Users\Saikat\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,categories,lat,lng
0,Axis Bank ATM,ATM,22.700560,88.322620
1,Axis Bank ATM,ATM,22.721077,88.324097
2,Axis Bank ATM,ATM,22.720780,88.331670


###### Explore nearest venues of all pincode areas

In [20]:
def get_nearby_venues(names,latitudes,longitudes,district,post_office):
    radius = 2000
    limit=100
    venues_list=[]
    for name,lat,lng,dst,po in zip(names,latitudes,longitudes,district,post_office):
        url="https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,radius,limit)
        results=requests.get(url).json()['response']['groups'][0]['items']
        venues_list.append([(name,lat,lng,dst,po,v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],v['venue']['categories'][0]['name'])for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Pincode','Local Latitude','Local Longitude','District','Post Office','Venue','Venue Latitude','Venue Longitude','Venue Category']
    return(nearby_venues)

In [21]:
kol_explore=get_nearby_venues(merge_kol_df['Pincode'],merge_kol_df['Latitude'],merge_kol_df['Longitude'],merge_kol_df['District'],merge_kol_df['Post Office'])
kol_explore

,Pincode,Local Latitude,Local Longitude,District,Post Office,Venue,Venue Latitude,Venue Longitude,Venue Category
0,700001,22.7101,88.3167,Kolkata,"Council House Street S.O, Customs House S.O, K...",Axis Bank ATM,22.700560,88.322620,ATM
1,700001,22.7101,88.3167,Kolkata,"Council House Street S.O, Customs House S.O, K...",Axis Bank ATM,22.721077,88.324097,ATM
2,700001,22.7101,88.3167,Kolkata,"Council House Street S.O, Customs House S.O, K...",Axis Bank ATM,22.720780,88.331670,ATM
3,700002,22.7101,88.3167,Kolkata,"Cossipore Gun Factory S.O, Cossipore H.O, Indi...",Axis Bank ATM,22.700560,88.322620,ATM
4,700002,22.7101,88.3167,Kolkata,"Cossipore Gun Factory S.O, Cossipore H.O, Indi...",Axis Bank ATM,22.721077,88.324097,ATM
...,...,...,...,...,...,...,...,...,...
839,700099,22.5553,88.3558,Kolkata,"Kalikapur S.O, Mukundapur S.O",Shoppers Stop,22.538354,88.351310,Department Store
840,700099,22.5553,88.3558,Kolkata,"Kalikapur S.O, Mukundapur S.O",E-Mall,22.566955,88.354874,Shopping Mall
841,700099,22.5553,88.3558,Kolkata,"Kalikapur S.O, Mukundapur S.O",Tandoori Delights,22.544762,88.364239,Mughlai Restaurant
842,700099,22.5553,88.3558,Kolkata,"Kalikapur S.O, Mukundapur S.O",HDFC Bank,22.539553,88.354840,Bank


###### Onehot coding

In [22]:
kol_onehot=pd.get_dummies(kol_explore['Venue Category'],prefix='',prefix_sep='')
kol_onehot['Pincode']=kol_explore['Pincode']
first_column=kol_onehot.pop('Pincode')
kol_onehot.insert(loc=0,column='Pincode',value=first_column)
kol_onehot.head()

,Pincode,ATM,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Astrologer,Awadhi Restaurant,BBQ Joint,Bakery,Bank,Beer Garden,Bengali Restaurant,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Brewery,Bus Stop,Business Service,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Construction & Landscaping,Cricket Ground,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market,Food,Food Court,Frozen Yogurt Shop,Gastropub,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Kerala Restaurant,Lounge,Market,Men's Store,Metro Station,Mexican Restaurant,Military Base,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Neighborhood,Nightclub,Park,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Plaza,Pool,Pub,Racetrack,Residential Building (Apartment / Condo),Restaurant,River,Sandwich Place,Scenic Lookout,Shoe Store,Shopping Mall,Snack Place,South Indian Restaurant,Sports Club,Stadium,Steakhouse,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theme Park,Theme Restaurant,Train Station,Vegetarian / Vegan Restaurant,Zoo
0,700001,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,700001,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,700001,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,700002,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,700002,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


###### Find the average number of venue per pincode area

In [23]:
kol_grouped=kol_onehot.groupby('Pincode').mean().reset_index()
kol_grouped

,Pincode,ATM,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Astrologer,Awadhi Restaurant,BBQ Joint,Bakery,Bank,Beer Garden,Bengali Restaurant,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Brewery,Bus Stop,Business Service,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Construction & Landscaping,Cricket Ground,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market,Food,Food Court,Frozen Yogurt Shop,Gastropub,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Kerala Restaurant,Lounge,Market,Men's Store,Metro Station,Mexican Restaurant,Military Base,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Neighborhood,Nightclub,Park,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Plaza,Pool,Pub,Racetrack,Residential Building (Apartment / Condo),Restaurant,River,Sandwich Place,Scenic Lookout,Shoe Store,Shopping Mall,Snack Place,South Indian Restaurant,Sports Club,Stadium,Steakhouse,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theme Park,Theme Restaurant,Train Station,Vegetarian / Vegan Restaurant,Zoo
0,700001,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,700002,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,700003,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,700004,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0000

###### Get the top 10 venues which are available in a pincode area

In [24]:
def return_top_venues(df,top):
    df_top=df.iloc[1:]
    df_top=df_top.sort_values(ascending=False)
    return df_top.index.values[0:top]

In [25]:
top_venues=10
indicators=['st','nd','rd']
columns=['Pincode']
for i in range(0,top_venues):
    try:
        columns.append('Top '+str(i+1)+indicators[i]+' Venue')
    except:
        columns.append('Top '+str(i+1)+'th Venue')
kol_sorted=pd.DataFrame(columns=columns)
kol_sorted['Pincode']=kol_grouped['Pincode']
for i in range(0,kol_grouped.shape[0]):
    kol_sorted.iloc[i,1:]=return_top_venues(kol_grouped.iloc[i,:],top_venues)
kol_sorted

,Pincode,Top 1st Venue,Top 2nd Venue,Top 3rd Venue,Top 4th Venue,Top 5th Venue,Top 6th Venue,Top 7th Venue,Top 8th Venue,Top 9th Venue,Top 10th Venue
0,700001,ATM,Frozen Yogurt Shop,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market,Food
1,700002,ATM,Frozen Yogurt Shop,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market,Food
2,700003,ATM,Frozen Yogurt Shop,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market,Food
3,700004,ATM,Frozen Yogurt Shop,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market,Food
4,700005,ATM,Frozen Yogurt Shop,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market,Food
5,700006,ATM,Pharmacy,Food Court,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market
6,700007,ATM,Frozen Yogurt Shop,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market,Food
7,700008,ATM,Frozen Yogurt Shop,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market,Food
8,700009,ATM,Frozen Yogurt Shop,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market,Food
9,700015,Chinese Restaurant,Indian Restaurant,Hotel,Coffee Shop,Mughlai Restaurant,Multicuisine Indian Restaurant,Brewery,Bengali Restaurant,Park,Asian Restaurant


###### Use KMeans clustering to cluster the areas

In [26]:
kclusters=5
kol_cluster=kol_grouped.drop('Pincode',1)
kmeans=KMeans(n_clusters=kclusters,random_state=0).fit(kol_cluster)
kmeans.labels_

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 4, 4, 4, 4, 1, 4, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 3, 3, 3, 3,
       1, 3, 3, 4, 4, 4, 4, 1])

###### Merging all information in a single table

In [27]:
kol_sorted.insert(0,'Cluster Labels',kmeans.labels_)
kol_merged=merge_kol_df
kol_merged=kol_sorted.join(kol_merged.set_index('Pincode'),on='Pincode')
col=kol_merged.columns.tolist()
col=col[-4:]+col[:-4]
kol_merged=kol_merged[col]
pincode=kol_merged.pop('Pincode')
kol_merged.insert(0,column='Pincode',value=pincode)
kol_merged

,Pincode,District,Post Office,Latitude,Longitude,Cluster Labels,Top 1st Venue,Top 2nd Venue,Top 3rd Venue,Top 4th Venue,Top 5th Venue,Top 6th Venue,Top 7th Venue,Top 8th Venue,Top 9th Venue,Top 10th Venue
0,700001,Kolkata,"Council House Street S.O, Customs House S.O, K...",22.7101,88.316700,2,ATM,Frozen Yogurt Shop,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market,Food
1,700002,Kolkata,"Cossipore Gun Factory S.O, Cossipore H.O, Indi...",22.7101,88.316700,2,ATM,Frozen Yogurt Shop,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market,Food
2,700003,Kolkata,"Amrita Bazar Partika S.O, Baghbazar S.O, Giris...",22.7101,88.316700,2,ATM,Frozen Yogurt Shop,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market,Food
3,700004,Kolkata,"R.G.Kar Medical College S.O, Shyambazar Mail S...",22.7101,88.316700,2,ATM,Frozen Yogurt Shop,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market,Food
4,700005,Kolkata,"Ahritola S.O, Hatkhola S.O",22.7101,88.316700,2,ATM,Frozen Yogurt Shop,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market,Food
5,700006,Kolkata,"Beadon Street S.O, Bidhan Sarani S.O, Jorasank...",22.7101,88.322286,2,ATM,Pharmacy,Food Court,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market
6,700007,Kolkata,"Archana S.O, Barabazar H.O, College Square S.O...",22.7101,88.316700,2,ATM,Frozen Yogurt Shop,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market,Food
7,700008,Kolkata,Barisha S.O,22.7101,88.316700,2,ATM,Frozen Yogurt Shop,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market,Food
8,700009,Kolkata,"Parsibagan S.O, Raja Ram Mohan Sarani S.O",22.7101,88.316700,2,ATM,Frozen Yogurt Shop,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market,Food
9,700015,Kolkata,"Sales Tax Building S.O, Seal Lane S.O, Tangra S.O",22.5500,88.383300,1,Chinese Restaurant,Indian Restaurant,Hotel,Coffee Shop,Mughlai Restaurant,Multicuisine Indian Restaurant,Brewery,Bengali Restaurant,Park,Asian Restaurant


###### Display the clustered area in a map

In [28]:
latitude=merge_kol_df.iloc[30]['Latitude']
longitude=merge_kol_df.iloc[30]['Longitude']
map_clusters=folium.Map(location=[latitude,longitude],zoom_start=11)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
for lat,lng,poi,cluster in zip(kol_merged['Latitude'],kol_merged['Longitude'],kol_merged['Pincode'],kol_merged['Cluster Labels']):
    label=folium.Popup(str(poi)+' cluster '+str(cluster),parse_html=True)
    folium.CircleMarker([lat,lng],radius=5,popup=label,color=rainbow[cluster-1],fill=True,fill_color=rainbow[cluster-1],fill_opacity=0.7).add_to(map_clusters)
map_clusters

###### Analysis of clusters

###### Cluster 1 - Total area covered under cluster 1 and the pincode of area where movie theater exists

In [30]:
cluster_one=kol_merged[kol_merged['Cluster Labels']==0]
cluster_one

,Pincode,District,Post Office,Latitude,Longitude,Cluster Labels,Top 1st Venue,Top 2nd Venue,Top 3rd Venue,Top 4th Venue,Top 5th Venue,Top 6th Venue,Top 7th Venue,Top 8th Venue,Top 9th Venue,Top 10th Venue
32,700061,Kolkata,"Jairampur S.O (Kolkata), Kasthadanga S.O, Sars...",22.4839,88.3038,0,ATM,Indian Restaurant,Health & Beauty Service,Indie Movie Theater,Theme Park,Coffee Shop,Cricket Ground,Department Store,Dessert Shop,Dhaba
33,700062,Kolkata,W.B.Governors Camp. S.O,22.4839,88.3038,0,ATM,Indian Restaurant,Health & Beauty Service,Indie Movie Theater,Theme Park,Coffee Shop,Cricket Ground,Department Store,Dessert Shop,Dhaba
35,700065,Kolkata,"Durganagar S.O, Health Institute S.O, Rabindra...",22.4839,88.3038,0,ATM,Indian Restaurant,Health & Beauty Service,Indie Movie Theater,Theme Park,Coffee Shop,Cricket Ground,Department Store,Dessert Shop,Dhaba
36,700066,Kolkata,Bidhangarh S.O,22.4839,88.3038,0,ATM,Indian Restaurant,Health & Beauty Service,Indie Movie Theater,Theme Park,Coffee Shop,Cricket Ground,Department Store,Dessert Shop,Dhaba
37,700067,Kolkata,"Lily Biscuit S.O, Ultadanga Main Road S.O",22.4839,88.3038,0,ATM,Indian Restaurant,Health & Beauty Service,Indie Movie Theater,Theme Park,Coffee Shop,Cricket Ground,Department Store,Dessert Shop,Dhaba
38,700068,Kolkata,Jodhpur Park S.O,22.4839,88.3038,0,ATM,Indian Restaurant,Health & Beauty Service,Indie Movie Theater,Theme Park,Coffee Shop,Cricket Ground,Department Store,Dessert Shop,Dhaba
39,700069,Kolkata,"Esplanade S.O, Income Tax Building S.O",22.4839,88.3038,0,ATM,Indian Restaurant,Health & Beauty Service,Indie Movie Theater,Theme Park,Coffee Shop,Cricket Ground,Department Store,Dessert Shop,Dhaba


In [31]:
for i in range(0,cluster_one.shape[0]):
    for j in range(0,10):
        if 'Movie' in cluster_one.iloc[i][j+6]:
            print(cluster_one.iloc[i]['Pincode'])

700061
700062
700065
700066
700067
700068
700069


###### Cluster 2 - Total area covered under cluster 2 and the pincode of area where movie theater exists

In [32]:
cluster_two=kol_merged[kol_merged['Cluster Labels']==1]
cluster_two

,Pincode,District,Post Office,Latitude,Longitude,Cluster Labels,Top 1st Venue,Top 2nd Venue,Top 3rd Venue,Top 4th Venue,Top 5th Venue,Top 6th Venue,Top 7th Venue,Top 8th Venue,Top 9th Venue,Top 10th Venue
9,700015,Kolkata,"Sales Tax Building S.O, Seal Lane S.O, Tangra S.O",22.5500,88.383300,1,Chinese Restaurant,Indian Restaurant,Hotel,Coffee Shop,Mughlai Restaurant,Multicuisine Indian Restaurant,Brewery,Bengali Restaurant,Park,Asian Restaurant
10,700019,Kolkata,"Ballygunge RS S.O, Ballygunge S.O, Ballygunge ...",22.5290,88.368000,1,Café,Chinese Restaurant,Hotel,Bengali Restaurant,Dhaba,Park,Bakery,Mughlai Restaurant,Vegetarian / Vegan Restaurant,Plaza
11,700022,Kolkata,"Bakery Road S.O, Hastings S.O",22.5500,88.333300,1,Hotel,Fast Food Restaurant,Tea Room,Italian Restaurant,Café,Indian Restaurant,Shopping Mall,Nightclub,Lounge,Dhaba
12,700024,Kolkata,"Garden Reach S.O, K.C.Mills S.O, P.G.Reach S.O...",22.5439,88.306700,1,Awadhi Restaurant,Market,Harbor / Marina,Construction & Landscaping,Zoo,Food Court,Department Store,Dessert Shop,Dhaba,Electronics Store
17,700034,Kolkata,"Behala Municipal Market S.O, Behala S.O, Brahm...",22.5553,88.291633,1,Astrologer,Harbor / Marina,River,Construction & Landscaping,Botanical Garden,Zoo,Food,Department Store,Dessert Shop,Dhaba
19,700036,Kolkata,"Baranagar S.O, Bengal Immunity S.O, Kuthighat ...",22.5553,88.370133,1,Café,Mughlai Restaurant,Indian Restaurant,Nightclub,Chinese Restaurant,Hotel,Bakery,Fast Food Restaurant,Pub,Restaurant
20,700037,Kolkata,"Belgachia Road S.O, Belgachia S.O, Northern Av...",22.6000,88.383300,1,Train Station,Indie Movie Theater,River,Bakery,Bengali Restaurant,Café,Indian Restaurant,Indian Sweet Shop,Market,Metro Station
21,700038,Kolkata,Sahapur S.O,22.5553,88.355800,1,Café,Nightclub,Mughlai Restaurant,Indian Restaurant,Hotel,Restaurant,Shopping Mall,Pub,Bakery,Department Store
22,700040,Kolkata,"Netaji Nagar S.O (Kolkata), Regent Park S.O, R...",22.5333,88.391700,1,Chinese Restaurant,Hotel,Indian Restaurant,Asian Restaurant,Train Station,Bengali Restaurant,Multicuisine Indian Restaurant,Coffee Shop,Mughlai Restaurant,Plaza
23,700041,Kolkata,"Paschim Putiari S.O, Sirity S.O",22.5333,88.391700,1,Chinese Restaurant,Hotel,Indian Restaurant,Asian Restaurant,Train Station,Bengali Restaurant,Multicuisine Indian Restaurant,Coffee Shop,Mughlai Restaurant,Plaza


In [33]:
for i in range(0,cluster_two.shape[0]):
    for j in range(0,10):
        if 'Movie' in cluster_two.iloc[i][j+6]:
            print(cluster_two.iloc[i]['Pincode'])

700037


###### Cluster 3 - Total area covered under cluster 3 and the pincode of area where movie theater exists

In [34]:
cluster_three=kol_merged[kol_merged['Cluster Labels']==2]
cluster_three

,Pincode,District,Post Office,Latitude,Longitude,Cluster Labels,Top 1st Venue,Top 2nd Venue,Top 3rd Venue,Top 4th Venue,Top 5th Venue,Top 6th Venue,Top 7th Venue,Top 8th Venue,Top 9th Venue,Top 10th Venue
0,700001,Kolkata,"Council House Street S.O, Customs House S.O, K...",22.7101,88.316700,2,ATM,Frozen Yogurt Shop,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market,Food
1,700002,Kolkata,"Cossipore Gun Factory S.O, Cossipore H.O, Indi...",22.7101,88.316700,2,ATM,Frozen Yogurt Shop,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market,Food
2,700003,Kolkata,"Amrita Bazar Partika S.O, Baghbazar S.O, Giris...",22.7101,88.316700,2,ATM,Frozen Yogurt Shop,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market,Food
3,700004,Kolkata,"R.G.Kar Medical College S.O, Shyambazar Mail S...",22.7101,88.316700,2,ATM,Frozen Yogurt Shop,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market,Food
4,700005,Kolkata,"Ahritola S.O, Hatkhola S.O",22.7101,88.316700,2,ATM,Frozen Yogurt Shop,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market,Food
5,700006,Kolkata,"Beadon Street S.O, Bidhan Sarani S.O, Jorasank...",22.7101,88.322286,2,ATM,Pharmacy,Food Court,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market
6,700007,Kolkata,"Archana S.O, Barabazar H.O, College Square S.O...",22.7101,88.316700,2,ATM,Frozen Yogurt Shop,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market,Food
7,700008,Kolkata,Barisha S.O,22.7101,88.316700,2,ATM,Frozen Yogurt Shop,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market,Food
8,700009,Kolkata,"Parsibagan S.O, Raja Ram Mohan Sarani S.O",22.7101,88.316700,2,ATM,Frozen Yogurt Shop,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant,Field,Flea Market,Food


In [35]:
for i in range(0,cluster_three.shape[0]):
    for j in range(0,10):
        if 'Movie' in cluster_three.iloc[i][j+6]:
            print(cluster_three.iloc[i]['Pincode'])

###### Cluster 4 - Total area covered under cluster 4 and the pincode of area where movie theater exists

In [36]:
cluster_four=kol_merged[kol_merged['Cluster Labels']==3]
cluster_four

,Pincode,District,Post Office,Latitude,Longitude,Cluster Labels,Top 1st Venue,Top 2nd Venue,Top 3rd Venue,Top 4th Venue,Top 5th Venue,Top 6th Venue,Top 7th Venue,Top 8th Venue,Top 9th Venue,Top 10th Venue
40,700071,Kolkata,"Little Russel Street S.O, Middleton Row S.O, R...",22.588,88.3185,3,Train Station,Business Service,Bank,Zoo,Food Court,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant
41,700072,Kolkata,"Hindustan Building S.O, Princep Street S.O, So...",22.588,88.3185,3,Train Station,Business Service,Bank,Zoo,Food Court,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant
42,700073,Kolkata,"Chittaranjan Avenue S.O (Kolkata), Colootola S...",22.588,88.3185,3,Train Station,Business Service,Bank,Zoo,Food Court,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant
43,700074,Kolkata,"Dumdum Road S.O, Jawpore S.O, Motijheel S.O (B...",22.588,88.3185,3,Train Station,Business Service,Bank,Zoo,Food Court,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant
45,700077,Kolkata,Bediapara S.O,22.588,88.3185,3,Train Station,Business Service,Bank,Zoo,Food Court,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant
46,700078,Kolkata,"Haltu S.O, Jadavgarh S.O",22.588,88.3185,3,Train Station,Business Service,Bank,Zoo,Food Court,Department Store,Dessert Shop,Dhaba,Electronics Store,Fast Food Restaurant


In [37]:
for i in range(0,cluster_four.shape[0]):
    for j in range(0,10):
        if 'Movie' in cluster_four.iloc[i][j+6]:
            print(cluster_four.iloc[i]['Pincode'])

###### Cluster 5 - Total area covered under cluster 5 and the pincode of area where movie theater exists

In [38]:
cluster_five=kol_merged[kol_merged['Cluster Labels']==4]
cluster_five

,Pincode,District,Post Office,Latitude,Longitude,Cluster Labels,Top 1st Venue,Top 2nd Venue,Top 3rd Venue,Top 4th Venue,Top 5th Venue,Top 6th Venue,Top 7th Venue,Top 8th Venue,Top 9th Venue,Top 10th Venue
13,700030,Kolkata,"Ashokegarh S.O, Bonhooghly S.O, Ghugudanga S.O...",22.5668,88.3016,4,ATM,Botanical Garden,Pizza Place,Train Station,Ice Cream Shop,Hotel,Cricket Ground,Department Store,Dessert Shop,Dhaba
14,700031,Kolkata,"Dhakuria S.O, K.P.Roy Lane S.O",22.5668,88.3016,4,ATM,Botanical Garden,Pizza Place,Train Station,Ice Cream Shop,Hotel,Cricket Ground,Department Store,Dessert Shop,Dhaba
15,700032,Kolkata,"Bijoygarh S.O, Jadavpur University S.O, Pgh Sh...",22.5668,88.3016,4,ATM,Botanical Garden,Pizza Place,Train Station,Ice Cream Shop,Hotel,Cricket Ground,Department Store,Dessert Shop,Dhaba
16,700033,Kolkata,"Indrani Park S.O, T.C.Road S.O, Tollygunge H.O",22.5668,88.3016,4,ATM,Botanical Garden,Pizza Place,Train Station,Ice Cream Shop,Hotel,Cricket Ground,Department Store,Dessert Shop,Dhaba
18,700035,Kolkata,"Alambazar S.O, Deshbandhu Road S.O",22.5668,88.3016,4,ATM,Botanical Garden,Pizza Place,Train Station,Ice Cream Shop,Hotel,Cricket Ground,Department Store,Dessert Shop,Dhaba
47,700090,Kolkata,"Netaji Colony S.O, Noapara S.O (Bankura)",22.4485,88.3883,4,ATM,Bank,Metro Station,Pizza Place,Hookah Bar,History Museum,Department Store,Dessert Shop,Dhaba,Electronics Store
48,700092,Kolkata,Regent Estate S.O,22.4485,88.3883,4,ATM,Bank,Metro Station,Pizza Place,Hookah Bar,History Museum,Department Store,Dessert Shop,Dhaba,Electronics Store
49,700094,Kolkata,"Panchasayar S.O, S.R.F.T.I. S.O",22.4485,88.3883,4,ATM,Bank,Metro Station,Pizza Place,Hookah Bar,History Museum,Department Store,Dessert Shop,Dhaba,Electronics Store
50,700095,Kolkata,Golf Green S.O,22.4485,88.3883,4,ATM,Bank,Metro Station,Pizza Place,Hookah Bar,History Museum,Department Store,Dessert Shop,Dhaba,Electronics Store


In [39]:
for i in range(0,cluster_five.shape[0]):
    for j in range(0,10):
        if 'Movie' in cluster_five.iloc[i][j+6]:
            print(cluster_five.iloc[i]['Pincode'])

###### Number of area in each cluster

In [40]:
cluster_one_size=cluster_one.shape[0]
cluster_two_size=cluster_two.shape[0]
cluster_three_size=cluster_three.shape[0]
cluster_four_size=cluster_four.shape[0]
cluster_five_size=cluster_five.shape[0]
print('No of areas in cluster_1 '+str(cluster_one_size)+'\nNo of areas in cluster_2 '+str(cluster_two_size)+'\nNo of areas in cluster_3 '+str(cluster_three_size)+'\nNo of areas in cluster_4 '+str(cluster_four_size)+'\nNo of areas in cluster_5 '+str(cluster_five_size))

No of areas in cluster_1 7
No of areas in cluster_2 21
No of areas in cluster_3 9
No of areas in cluster_4 6
No of areas in cluster_5 9


### Conclusion

The total number of areas in cluster_1 and the total number of movie theater in this cluster is same which is 7.It seems if a company starts its new business in any area on cluster_1 it will be very competitive and difficult to get the success.Cluster_2 has total 21 areas and only one movie theater exists.Cluster_3, Cluster_4 and Cluster_5 has no movie theater but the number of area covered are 9,6,9 respectively.The size of cluster_2 is almost 2.5 times of cluster_3 and cluster_5.Big cluster size means population is also very high, hence though cluster_2 has one movie theater it will be good to start a new theater in this cluster if we also consider cluster size and population is proportional.Second choice will be cluster_3 or cluster_5 as no movie theater exists and size is also comparatively bigger than cluster_4.Third or last choice will be cluster_4 which contains least number of area.Cluster_1 is not recommended due to high level of competition.